In [ ]:
%pip install git+https://github.com/gbdrt/mu-ppl

In [ ]:
from mu_ppl import *
import matplotlib.pyplot as plt
import numpy as np

# Introduction to probabilistic programming with mu-ppl

G. Baudart & C. Tasson


## Random execution with `sample`

A probabilistic model is a classic Python function.
The probabilistic operator `sample` draws a sample from a distribution which yields non-deterministic behavior.
For instance, what is the distribution described by the following program ?

In [ ]:
def dice() -> int:
    a = sample(RandInt(1, 6), name="a")
    b = sample(RandInt(1, 6), name="b")
    return a + b

We can already execute the model to get a list of random samples.

In [ ]:
[dice() for _ in range(10)]

## Computing a distribution with `Infer`

The `infer` operator is used to compute the distribution describes by a model.
Following [Pyro](https://pyro.ai/), in mu-ppl we use _context managers_ to implement different inference algorithms.

Our model, only use finite discrete support distribution.
We can enumerate all possible execution to compute the distribution.

In [ ]:
with Enumeration():
    dist: Categorical[int] = infer(dice)
    viz(dist)

**Remark:** In the model definition, each `sample` operator takes an optional argument `name` which specify a _unique_ name for each sample site.
This argument is required to run some inference algorithm.
In the enumation algorithm we use the names to compute the execution paths to be explored.

## Distributions

In mu-ppl a distribution `d` is an object which implement the following methods:
- `d.sample()` draws a random sample
- `d.logprob(v)` computes the log-probability of `d` at `v`
- `d.stats()` returns the mean and variance.

In [ ]:
norm = Gaussian(0, 1)
print(f"A sample from N(0, 1): {norm.sample()}")
print(f"mu, sigma = {norm.stats()}")

x = np.linspace(-3, 3, 100) 
plt.plot(x, np.exp(norm.log_prob(x)))

## Conditionning the model with `assume`

We can also make assumptions that must always hold.
For instance, what is the distribution modeled by the following program?

In [ ]:
def hard_dice() -> int:
    a = sample(RandInt(1, 6), name="a")
    b = sample(RandInt(1, 6), name="b")
    assume (a != b)
    return a + b

The `assume` operator "skews" the distribution.

In [ ]:
with Enumeration():
    dist: Categorical[int] = infer(hard_dice)
    print(dist.stats())
    viz(dist)

## Soft conditionning with `factor`

The `assume` operator performs _hard conditionning_ declaring an assertion that must hold for all valid execution.
An alternative is to rely on _soft conditionning_ which associates a weigth to each execution.

For instance, we can modify our first `dice` model to sligthly favor executions where the two dice return different values.
Again the resulting distribution is skewed.

In [ ]:
def dice() -> int:
    a = sample(RandInt(1, 6), name="a")
    b = sample(RandInt(1, 6), name="b")
    factor(1 if a != b else -0.5)
    return a + b

with Enumeration():
    dist: Categorical[float] = infer(dice)  # type: ignore
    viz(dist)